# BipartiteGraph class definition

In this part we define a class called `BipartiteGraph` which will be used to:
- compute and print all the maximum matchings;
- compute and print the maximal independent sets we keep over time using our heuristic;
- show how fast the maximal independent set chosen using our heuristic drop in size as time progresses compared to all the other maximum matching in the bipartite graph.

A small note we have to make is that the adjacency matrix $A$ of a bipartite graph $(L, R; E)$ of size $\ell$ and $r$ can be written in the form:

\begin{align*}
A = \begin{bmatrix}
    0_{\ell \times \ell} & B         \\ 
    B^T             & 0_{r \times r} \\
\end{bmatrix}
\end{align*}

In order to save space, We decided to keep just $B$ in memory.

In [1]:
from itertools import combinations

In [2]:
class BipartiteGraph:
    def __init__(self, num_categories, num_points):
        # Support counters
        self.num_points = num_points
        self.num_categories = num_categories
        self.num_edges = 0
        
        # These structures are used to have named vertices in the graph (both points and categories)
        self.points = {}
        self.points_names = [''] * num_points

        self.categories = {}
        self.categories_names = [''] * num_categories
        
        # The adjacency matrix needs to takes into account only the edges between points and categories 
        # It is a matrix of size num_categories X num_points
        self.adj_matrix = [[0] * num_points for _ in range(num_categories)]


    def add_edge(self, edge_category, edge_point):
        # If the name of the point or category doesn't exist try to add it
        if (edge_point not in self.points):
            if (len(self.points) == self.num_points):
                raise IndexError('You cannot have more than {} points'.format(self.num_points))
            self.points_names[len(self.points)] = edge_point 
            self.points[edge_point] = len(self.points)
        
        if (edge_category not in self.categories):
            if (len(self.categories) == self.num_categories):
                raise IndexError('You cannot have more than {} categories'.format(self.num_categories))
            self.categories_names[len(self.categories)] = edge_category 
            self.categories[edge_category] = len(self.categories)

        # Add the edge to the adjacency matrix and increase the number of edges
        index_point = self.points[edge_point]
        index_category = self.categories[edge_category]

        self.adj_matrix[index_category][index_point] = 1
        self.num_edges += 1


    def get_adjacency_matrix(self):
        return self.adj_matrix


    def get_edges_list(self):
        # Compute both the lists of edges using indices and using names  
        result_list = []
        result_list_names = []
        
        for index_category in range(self.num_categories):
           for index_point in range(self.num_points):
                if (self.adj_matrix[index_category][index_point] == 1):
                    result_list.append((index_category, index_point))
                    result_list_names.append((self.categories_names[index_category], self.points_names[index_point]))

        return (result_list, result_list_names)
    

    def _is_subset_edges_matching(self, edge_list):
        # In order to check if a subset of edges is a matching, it is sufficient to check that each node (point of category) is connected to at most one edge in the subset
        degree_points = [0 for ixx in range(self.num_points)]
        degree_categories = [0 for ixx in range(self.num_categories)]

        for index_category, index_point in edge_list:
            degree_points[index_point] += 1
            degree_categories[index_category] += 1
        
        for point_degree in degree_points:
            if (point_degree > 1):
                del degree_points
                del degree_categories
                return 0

        for category_degree in degree_categories:
            if (category_degree > 1):
                del degree_points
                del degree_categories
                return 0

        del degree_points
        del degree_categories
        return 1

    def print_all_matchings(self, min_size=1, max_size=None, verbose=True):
        # First let's check that the parameters given are not out of bound
        if (max_size == None):
            max_size = min(self.num_edges, self.num_points, self.num_categories)
            
        if (max_size < 1 or max_size > min(self.num_edges, self.num_points, self.num_categories)):
            raise ValueError('The "max_size" parameter is out of range')
        if (min_size < 1 or min_size > max_size):
            raise ValueError('The "min_size" parameter is out of range')
        if (verbose is not True and verbose is not False):
            raise ValueError('The "verbose" parameter must be a boolean value.')

        (support_edge_list, support_edge_list_names) = self.get_edges_list()
        #print('The complete list of edges is {}, with indices {}'.format(support_edge_list_names, support_edge_list))
        print('The complete list of edges is {}'.format(support_edge_list_names))
        print()
        
        for current_size in range(max_size, min_size-1, -1):
            print('Searching matchings of size: {}'.format(current_size))

            num_matching_of_size = 0
            best_endpoints_lists_names = []
            best_endpoints_lists = []
            best_endpoints_list_survives_until = [0 for ixx in range(self.num_points)]

            combinations_list = list(combinations(range(self.num_edges), current_size))
            print('There are {} combinations to check'.format(len(combinations_list)))

            for combination in combinations_list:
                # Construct the sublist of edges 
                temp_edge_list = []
                temp_edge_list_names = []

                for item in combination:
                    temp_edge_list.append(support_edge_list[item])
                    temp_edge_list_names.append(support_edge_list_names[item])

                # If it is a matching print some information
                if (True == self._is_subset_edges_matching(temp_edge_list)):
                    temp_list_survives_until = [0 for ixx in range(self.num_points)]
                    
                    # Compute how fast this matching drop in size as time progresses
                    for time_step in range (self.num_points):
                        for item_category, item_point in temp_edge_list:
                            if (item_point >= time_step):
                                temp_list_survives_until[time_step] += 1

                    # Keep the "best" matchings, meaning the ones that drop in size later than everybody else
                    best_flags = [1, 1] 
                    # flags[0] == 1 for 'is temp_edge_list at least as good as best_endpoints_lists'
                    # flags[1] == 0 for 'is temp_edge_list better than best_endpoints_lists'
                    
                    for time_step in range (0, self.num_points):
                        if (temp_list_survives_until[time_step] < best_endpoints_list_survives_until[time_step]):
                            best_flags[0] = 0
                        if (temp_list_survives_until[time_step] > best_endpoints_list_survives_until[time_step]):
                            best_flags[1] = 0

                    if (best_flags[0] == 1 and best_flags[1] == 0):
                        best_endpoints_list_survives_until = temp_list_survives_until
                        best_endpoints_lists_names = [temp_edge_list_names]
                        best_endpoints_lists = [temp_edge_list]

                    if (best_flags[0] == 1 and best_flags[1] == 1):
                        best_endpoints_lists_names.append(temp_edge_list_names)
                        best_endpoints_lists.append(temp_edge_list)

                    num_matching_of_size += 1
                    if(True == verbose):
                        print('The sublist of edges {} is a matching of size {}, its size decreases in time as follows: {}'.format(temp_edge_list_names, current_size, temp_list_survives_until))
                    
                del temp_edge_list
                del temp_edge_list_names
                
            del combinations_list

            if (num_matching_of_size > 0): 
                print()
                
                print('number of matchings of size {}: {}'.format(current_size, num_matching_of_size))
                print()
                
                print('the best matchings are:')
                for index_best_list in range(len(best_endpoints_lists_names)):
                    #print('{} {}'.format(best_endpoints_lists_names[index_best_list], best_endpoints_lists[index_best_list]))
                    print('{}'.format(best_endpoints_lists_names[index_best_list]))
                print('they descrease in size as follows {}'.format(best_endpoints_list_survives_until))

                del best_endpoints_lists
                del best_endpoints_lists_names
                break


    def _is_points_subset_leftendpoints_matching(self, points_mask, size):
        # Since we assume the number of edges from each point to be constant, we can say that the |support_edge_list| = O(size)
        support_edge_list = []
        support_edge_list_names = []
        (temp_edge_list, temp_edge_list_names) = self.get_edges_list()

        for index in range(len(temp_edge_list)):
            
            if (points_mask[temp_edge_list[index][1]] == 1):
                support_edge_list.append(temp_edge_list[index])
                support_edge_list_names.append(temp_edge_list_names[index])

        # Still, the number of combinations could be very high: 
        # num_combinations = O(|support_edge_list|! / ((|support_edge_list| - size)! * size!)
        # We assume anyway the number to be not too high in nominal terms given that we use this method only with instances of limited size
        # This is not the most efficient way of checking, we could use Hopcroft-Karp on the bipartite graph but it would require too much time to implement
        combinations_list = list(combinations(range(len(support_edge_list)), size))
        
        for combination in combinations_list:
            temp_edge_list = []
            temp_edge_list_names = []

            for item in combination:
                temp_edge_list.append(support_edge_list[item])
                temp_edge_list_names.append(support_edge_list_names[item])

            if (True == self._is_subset_edges_matching(temp_edge_list)):
                del support_edge_list
                del support_edge_list_names
                del combinations_list
                return temp_edge_list_names
        
        return None
    
    def print_heuristic_solution_over_time(self):       
        current_size = 0 
        possible_matching = []
        chosen_points = [0 for ixx in range(self.num_points)]

        # Simulate base case + inductive case 1
        for time_step in range(self.num_points):
            chosen_points[time_step] = 1
            current_size += 1

            possible_matching = self._is_points_subset_leftendpoints_matching(chosen_points, current_size)
            if (possible_matching == None):
                for index in range(self.num_points):
                    if (chosen_points[index] == 0):
                        continue
                    else:
                        chosen_points[index] = 0
                        current_size -= 1

                        possible_matching = self._is_points_subset_leftendpoints_matching(chosen_points, current_size)
                        if (possible_matching != None):
                            break

                        chosen_points[index] = 1
                        current_size += 1

            # Simulate inductive case 2, as if inductive case 1 ended at time 'time_step-1'
            chosen_points_survives_until = []
            current_sum = 0
            
            for num in reversed(chosen_points):
                current_sum += num
                chosen_points_survives_until.insert(0, current_sum)
            
            chosen_points_names = []

            for index in range(self.num_points):
                if (chosen_points[index] == 1):
                    chosen_points_names.append(self.points_names[index])

            print('At time step {} the set of {} points chosen is {} and a possible matching with these left endpoints is {}'.format(time_step, current_size, chosen_points_names, possible_matching))
            print('The independent set will decrease in size as follows {}'.format(chosen_points_survives_until))

            del chosen_points_names
            del chosen_points_survives_until
            del possible_matching
        
        del chosen_points


    def print_empirical_study(self, max_size=None, verbose=True):
        # First let's check that the parameter given is not out of bound
        if (max_size == None):
            max_size = min(self.num_edges, self.num_points, self.num_categories)

        if (max_size < 1 or max_size > min(self.num_edges, self.num_points, self.num_categories)):
            raise ValueError('The max_size is out of range')
        if (verbose is not True and verbose is not False):
            raise ValueError('The "verbose" parameter must be a boolean value.')

        print('The adjacency matrix for the bipartite graph is (rows->categories, columns->points):')
        for row in self.adj_matrix:
            print(row)
        print()

        print('The maximum matchings for this bipartite graphs are:')
        self.print_all_matchings(max_size=max_size, verbose=verbose)
        print()

        print('As time progresses, the maximal independent set we keep using the heuristic is:')
        self.print_heuristic_solution_over_time()
        print()


# Empirical tests

Now that the class `BipartiteGraph` has been completely specified, 
we are ready to try it with a list of heterogeneous instances of bipartite graphs.

As we can see in the various cells below, our heuristic actually works in practice, given it always returns a subset of points representing the left endpoints of a maximal matching in the bipartite graph, even as time advances and the window closes.

In [3]:
#Example 1
bipartite_graph = BipartiteGraph(num_categories=2, num_points=3)
bipartite_graph.add_edge('A1', 'X1')
bipartite_graph.add_edge('A1', 'X2')

bipartite_graph.add_edge('A2', 'X1')
bipartite_graph.add_edge('A2', 'X2')
bipartite_graph.add_edge('A2', 'X3')

bipartite_graph.print_empirical_study()
del bipartite_graph

The adjacency matrix for the bipartite graph is (rows->categories, columns->points):
[1, 1, 0]
[1, 1, 1]

The maximum matchings for this bipartite graphs are:
The complete list of edges is [('A1', 'X1'), ('A1', 'X2'), ('A2', 'X1'), ('A2', 'X2'), ('A2', 'X3')]

Searching matchings of size: 2
There are 10 combinations to check
The sublist of edges [('A1', 'X1'), ('A2', 'X2')] is a matching of size 2, its size decreases in time as follows: [2, 1, 0]
The sublist of edges [('A1', 'X1'), ('A2', 'X3')] is a matching of size 2, its size decreases in time as follows: [2, 1, 1]
The sublist of edges [('A1', 'X2'), ('A2', 'X1')] is a matching of size 2, its size decreases in time as follows: [2, 1, 0]
The sublist of edges [('A1', 'X2'), ('A2', 'X3')] is a matching of size 2, its size decreases in time as follows: [2, 2, 1]

number of matchings of size 2: 4

the best matchings are:
[('A1', 'X2'), ('A2', 'X3')]
they descrease in size as follows [2, 2, 1]

As time progresses, the maximal independent 

In [4]:
#Example 2
bipartite_graph = BipartiteGraph(2, 5)
bipartite_graph.add_edge('A1', 'X1')
bipartite_graph.add_edge('A1', 'X2')
bipartite_graph.add_edge('A1', 'X3')

bipartite_graph.add_edge('A2', 'X3')
bipartite_graph.add_edge('A2', 'X4')
bipartite_graph.add_edge('A2', 'X5')

bipartite_graph.print_empirical_study()
del bipartite_graph

The adjacency matrix for the bipartite graph is (rows->categories, columns->points):
[1, 1, 1, 0, 0]
[0, 0, 1, 1, 1]

The maximum matchings for this bipartite graphs are:
The complete list of edges is [('A1', 'X1'), ('A1', 'X2'), ('A1', 'X3'), ('A2', 'X3'), ('A2', 'X4'), ('A2', 'X5')]

Searching matchings of size: 2
There are 15 combinations to check
The sublist of edges [('A1', 'X1'), ('A2', 'X3')] is a matching of size 2, its size decreases in time as follows: [2, 1, 1, 0, 0]
The sublist of edges [('A1', 'X1'), ('A2', 'X4')] is a matching of size 2, its size decreases in time as follows: [2, 1, 1, 1, 0]
The sublist of edges [('A1', 'X1'), ('A2', 'X5')] is a matching of size 2, its size decreases in time as follows: [2, 1, 1, 1, 1]
The sublist of edges [('A1', 'X2'), ('A2', 'X3')] is a matching of size 2, its size decreases in time as follows: [2, 2, 1, 0, 0]
The sublist of edges [('A1', 'X2'), ('A2', 'X4')] is a matching of size 2, its size decreases in time as follows: [2, 2, 1, 1, 

In [5]:
#Example 3
bipartite_graph = BipartiteGraph(4, 5)
bipartite_graph.add_edge('A1', 'X1')
bipartite_graph.add_edge('A1', 'X3')

bipartite_graph.add_edge('A2', 'X2')

bipartite_graph.add_edge('A3', 'X2')

bipartite_graph.add_edge('A4', 'X4')
bipartite_graph.add_edge('A4', 'X5')

bipartite_graph.print_empirical_study()
del bipartite_graph

The adjacency matrix for the bipartite graph is (rows->categories, columns->points):
[1, 1, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 1, 1]

The maximum matchings for this bipartite graphs are:
The complete list of edges is [('A1', 'X1'), ('A1', 'X3'), ('A2', 'X2'), ('A3', 'X2'), ('A4', 'X4'), ('A4', 'X5')]

Searching matchings of size: 4
There are 15 combinations to check
Searching matchings of size: 3
There are 20 combinations to check
The sublist of edges [('A1', 'X1'), ('A2', 'X2'), ('A4', 'X4')] is a matching of size 3, its size decreases in time as follows: [3, 2, 2, 1, 0]
The sublist of edges [('A1', 'X1'), ('A2', 'X2'), ('A4', 'X5')] is a matching of size 3, its size decreases in time as follows: [3, 2, 2, 1, 1]
The sublist of edges [('A1', 'X1'), ('A3', 'X2'), ('A4', 'X4')] is a matching of size 3, its size decreases in time as follows: [3, 2, 2, 1, 0]
The sublist of edges [('A1', 'X1'), ('A3', 'X2'), ('A4', 'X5')] is a matching of size 3, its size decreases in time a

In [6]:
#Example 4
bipartite_graph = BipartiteGraph(4, 5)
bipartite_graph.add_edge('A1', 'X1')
bipartite_graph.add_edge('A1', 'X2')
bipartite_graph.add_edge('A1', 'X3')
bipartite_graph.add_edge('A1', 'X4')

bipartite_graph.add_edge('A2', 'X3')

bipartite_graph.add_edge('A3', 'X3')
bipartite_graph.add_edge('A3', 'X5')

bipartite_graph.add_edge('A4', 'X4')
bipartite_graph.add_edge('A4', 'X5')

bipartite_graph.print_empirical_study()
del bipartite_graph

The adjacency matrix for the bipartite graph is (rows->categories, columns->points):
[1, 1, 1, 1, 0]
[0, 0, 1, 0, 0]
[0, 0, 1, 0, 1]
[0, 0, 0, 1, 1]

The maximum matchings for this bipartite graphs are:
The complete list of edges is [('A1', 'X1'), ('A1', 'X2'), ('A1', 'X3'), ('A1', 'X4'), ('A2', 'X3'), ('A3', 'X3'), ('A3', 'X5'), ('A4', 'X4'), ('A4', 'X5')]

Searching matchings of size: 4
There are 126 combinations to check
The sublist of edges [('A1', 'X1'), ('A2', 'X3'), ('A3', 'X5'), ('A4', 'X4')] is a matching of size 4, its size decreases in time as follows: [4, 3, 3, 2, 1]
The sublist of edges [('A1', 'X2'), ('A2', 'X3'), ('A3', 'X5'), ('A4', 'X4')] is a matching of size 4, its size decreases in time as follows: [4, 4, 3, 2, 1]

number of matchings of size 4: 2

the best matchings are:
[('A1', 'X2'), ('A2', 'X3'), ('A3', 'X5'), ('A4', 'X4')]
they descrease in size as follows [4, 4, 3, 2, 1]

As time progresses, the maximal independent set we keep using the heuristic is:
At time s

In [7]:
#Example 5
bipartite_graph = BipartiteGraph(6, 7)
bipartite_graph.add_edge('A1', 'X1')
bipartite_graph.add_edge('A1', 'X2')
bipartite_graph.add_edge('A1', 'X7')

bipartite_graph.add_edge('A2', 'X1')
bipartite_graph.add_edge('A2', 'X2')
bipartite_graph.add_edge('A2', 'X7')

bipartite_graph.add_edge('A3', 'X2')
bipartite_graph.add_edge('A3', 'X3')
bipartite_graph.add_edge('A3', 'X7')

bipartite_graph.add_edge('A4', 'X3')
bipartite_graph.add_edge('A4', 'X4')

bipartite_graph.add_edge('A5', 'X3')
bipartite_graph.add_edge('A5', 'X5')
bipartite_graph.add_edge('A5', 'X7')

bipartite_graph.add_edge('A6', 'X4')
bipartite_graph.add_edge('A6', 'X5')
bipartite_graph.add_edge('A6', 'X6')

bipartite_graph.print_empirical_study(verbose=False)
del bipartite_graph

The adjacency matrix for the bipartite graph is (rows->categories, columns->points):
[1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0]
[0, 1, 1, 1, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 1, 1, 0, 1, 0]
[0, 0, 0, 0, 1, 1, 1]

The maximum matchings for this bipartite graphs are:
The complete list of edges is [('A1', 'X1'), ('A1', 'X2'), ('A1', 'X7'), ('A2', 'X1'), ('A2', 'X2'), ('A2', 'X7'), ('A3', 'X2'), ('A3', 'X7'), ('A3', 'X3'), ('A4', 'X3'), ('A4', 'X4'), ('A5', 'X7'), ('A5', 'X3'), ('A5', 'X5'), ('A6', 'X4'), ('A6', 'X5'), ('A6', 'X6')]

Searching matchings of size: 6
There are 12376 combinations to check

number of matchings of size 6: 30

the best matchings are:
[('A1', 'X2'), ('A2', 'X7'), ('A3', 'X3'), ('A4', 'X4'), ('A5', 'X5'), ('A6', 'X6')]
[('A1', 'X7'), ('A2', 'X2'), ('A3', 'X3'), ('A4', 'X4'), ('A5', 'X5'), ('A6', 'X6')]
they descrease in size as follows [6, 6, 5, 4, 3, 2, 1]

As time progresses, the maximal independent set we keep using the heuristic is:
At time step 0 the set 

In [8]:
#Example 6
bipartite_graph = BipartiteGraph(6, 10)
bipartite_graph.add_edge('A1', 'X1')
bipartite_graph.add_edge('A1', 'X5')

bipartite_graph.add_edge('A2', 'X2')
bipartite_graph.add_edge('A2', 'X3')
bipartite_graph.add_edge('A2', 'X4')

bipartite_graph.add_edge('A3', 'X5')
bipartite_graph.add_edge('A3', 'X6')

bipartite_graph.add_edge('A4', 'X5')
bipartite_graph.add_edge('A4', 'X6')
bipartite_graph.add_edge('A4', 'X7')

bipartite_graph.add_edge('A5', 'X8')
bipartite_graph.add_edge('A5', 'X9')

bipartite_graph.add_edge('A6', 'X8')
bipartite_graph.add_edge('A6', 'X9')
bipartite_graph.add_edge('A6', 'X10')

bipartite_graph.print_empirical_study(verbose=False)
del bipartite_graph

The adjacency matrix for the bipartite graph is (rows->categories, columns->points):
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1]

The maximum matchings for this bipartite graphs are:
The complete list of edges is [('A1', 'X1'), ('A1', 'X5'), ('A2', 'X2'), ('A2', 'X3'), ('A2', 'X4'), ('A3', 'X5'), ('A3', 'X6'), ('A4', 'X5'), ('A4', 'X6'), ('A4', 'X7'), ('A5', 'X8'), ('A5', 'X9'), ('A6', 'X8'), ('A6', 'X9'), ('A6', 'X10')]

Searching matchings of size: 6
There are 5005 combinations to check

number of matchings of size 6: 60

the best matchings are:
[('A1', 'X5'), ('A2', 'X4'), ('A3', 'X6'), ('A4', 'X7'), ('A5', 'X9'), ('A6', 'X10')]
they descrease in size as follows [6, 6, 5, 5, 5, 4, 3, 2, 2, 1]

As time progresses, the maximal independent set we keep using the heuristic is:
At time step 0 the set of 1 points chosen is ['X1'] and a possible match

In [9]:
#Example 7
bipartite_graph = BipartiteGraph(num_categories=6, num_points=5)
bipartite_graph.add_edge('A1', 'X1')
bipartite_graph.add_edge('A1', 'X2')

bipartite_graph.add_edge('A2', 'X2')
bipartite_graph.add_edge('A2', 'X3')

bipartite_graph.add_edge('A3', 'X3')
bipartite_graph.add_edge('A3', 'X4')
bipartite_graph.add_edge('A3', 'X5')

bipartite_graph.add_edge('A4', 'X1')
bipartite_graph.add_edge('A4', 'X3')

bipartite_graph.add_edge('A5', 'X2')
bipartite_graph.add_edge('A5', 'X4')

bipartite_graph.add_edge('A6', 'X3')
bipartite_graph.add_edge('A6', 'X4')

bipartite_graph.print_empirical_study()
del bipartite_graph

The adjacency matrix for the bipartite graph is (rows->categories, columns->points):
[1, 1, 0, 0, 0]
[0, 1, 1, 0, 0]
[0, 0, 1, 1, 1]
[1, 0, 1, 0, 0]
[0, 1, 0, 1, 0]
[0, 0, 1, 1, 0]

The maximum matchings for this bipartite graphs are:
The complete list of edges is [('A1', 'X1'), ('A1', 'X2'), ('A2', 'X2'), ('A2', 'X3'), ('A3', 'X3'), ('A3', 'X4'), ('A3', 'X5'), ('A4', 'X1'), ('A4', 'X3'), ('A5', 'X2'), ('A5', 'X4'), ('A6', 'X3'), ('A6', 'X4')]

Searching matchings of size: 5
There are 1287 combinations to check
The sublist of edges [('A1', 'X1'), ('A2', 'X2'), ('A3', 'X5'), ('A4', 'X3'), ('A5', 'X4')] is a matching of size 5, its size decreases in time as follows: [5, 4, 3, 2, 1]
The sublist of edges [('A1', 'X1'), ('A2', 'X2'), ('A3', 'X5'), ('A4', 'X3'), ('A6', 'X4')] is a matching of size 5, its size decreases in time as follows: [5, 4, 3, 2, 1]
The sublist of edges [('A1', 'X1'), ('A2', 'X2'), ('A3', 'X5'), ('A5', 'X4'), ('A6', 'X3')] is a matching of size 5, its size decreases in 

In [10]:
#Example 8
bipartite_graph = BipartiteGraph(4, 12)
bipartite_graph.add_edge('A1', 'X1')
bipartite_graph.add_edge('A1', 'X2')
bipartite_graph.add_edge('A1', 'X3')
bipartite_graph.add_edge('A1', 'X4')
bipartite_graph.add_edge('A1', 'X9')

bipartite_graph.add_edge('A2', 'X5')
bipartite_graph.add_edge('A2', 'X6')
bipartite_graph.add_edge('A2', 'X7')
bipartite_graph.add_edge('A2', 'X8')
bipartite_graph.add_edge('A2', 'X9')

bipartite_graph.add_edge('A3', 'X9')
bipartite_graph.add_edge('A3', 'X10')
bipartite_graph.add_edge('A3', 'X11')
bipartite_graph.add_edge('A3', 'X12')

bipartite_graph.add_edge('A4', 'X11')
bipartite_graph.add_edge('A4', 'X12')

bipartite_graph.print_empirical_study(verbose=False)
del bipartite_graph

The adjacency matrix for the bipartite graph is (rows->categories, columns->points):
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]

The maximum matchings for this bipartite graphs are:
The complete list of edges is [('A1', 'X1'), ('A1', 'X2'), ('A1', 'X3'), ('A1', 'X4'), ('A1', 'X9'), ('A2', 'X9'), ('A2', 'X5'), ('A2', 'X6'), ('A2', 'X7'), ('A2', 'X8'), ('A3', 'X9'), ('A3', 'X10'), ('A3', 'X11'), ('A3', 'X12'), ('A4', 'X11'), ('A4', 'X12')]

Searching matchings of size: 4
There are 1820 combinations to check

number of matchings of size 4: 128

the best matchings are:
[('A1', 'X9'), ('A2', 'X8'), ('A3', 'X11'), ('A4', 'X12')]
[('A1', 'X9'), ('A2', 'X8'), ('A3', 'X12'), ('A4', 'X11')]
they descrease in size as follows [4, 4, 4, 4, 4, 3, 3, 3, 3, 2, 2, 1]

As time progresses, the maximal independent set we keep using the heuristic is:
At time step 0 the set of 1 points chosen is ['X1'] a

In [11]:
#Example 9
bipartite_graph = BipartiteGraph(5, 5)
bipartite_graph.add_edge('A1', 'X1')
bipartite_graph.add_edge('A1', 'X2')
bipartite_graph.add_edge('A1', 'X5')

bipartite_graph.add_edge('A2', 'X3')

bipartite_graph.add_edge('A3', 'X3')

bipartite_graph.add_edge('A4', 'X4')

bipartite_graph.add_edge('A5', 'X4')

bipartite_graph.print_empirical_study()
del bipartite_graph

The adjacency matrix for the bipartite graph is (rows->categories, columns->points):
[1, 1, 1, 0, 0]
[0, 0, 0, 1, 0]
[0, 0, 0, 1, 0]
[0, 0, 0, 0, 1]
[0, 0, 0, 0, 1]

The maximum matchings for this bipartite graphs are:
The complete list of edges is [('A1', 'X1'), ('A1', 'X2'), ('A1', 'X5'), ('A2', 'X3'), ('A3', 'X3'), ('A4', 'X4'), ('A5', 'X4')]

Searching matchings of size: 5
There are 21 combinations to check
Searching matchings of size: 4
There are 35 combinations to check
Searching matchings of size: 3
There are 35 combinations to check
The sublist of edges [('A1', 'X1'), ('A2', 'X3'), ('A4', 'X4')] is a matching of size 3, its size decreases in time as follows: [3, 2, 2, 2, 1]
The sublist of edges [('A1', 'X1'), ('A2', 'X3'), ('A5', 'X4')] is a matching of size 3, its size decreases in time as follows: [3, 2, 2, 2, 1]
The sublist of edges [('A1', 'X1'), ('A3', 'X3'), ('A4', 'X4')] is a matching of size 3, its size decreases in time as follows: [3, 2, 2, 2, 1]
The sublist of edges 

In [12]:
#Example 10
bipartite_graph = BipartiteGraph(10, 18)
bipartite_graph.add_edge('A1', 'X1')
bipartite_graph.add_edge('A1', 'X2')
bipartite_graph.add_edge('A1', 'X7')
bipartite_graph.add_edge('A1', 'X12')

bipartite_graph.add_edge('A2', 'X2')
bipartite_graph.add_edge('A2', 'X4')
bipartite_graph.add_edge('A2', 'X5')

bipartite_graph.add_edge('A3', 'X5')

bipartite_graph.add_edge('A4', 'X5')

bipartite_graph.add_edge('A5', 'X6')
bipartite_graph.add_edge('A5', 'X7')

bipartite_graph.add_edge('A6', 'X8')
bipartite_graph.add_edge('A6', 'X14')
bipartite_graph.add_edge('A6', 'X17')

bipartite_graph.add_edge('A7', 'X9')
bipartite_graph.add_edge('A7', 'X10')
bipartite_graph.add_edge('A7', 'X11')
bipartite_graph.add_edge('A7', 'X12')

bipartite_graph.add_edge('A8', 'X10')

bipartite_graph.add_edge('A9', 'X13')
bipartite_graph.add_edge('A9', 'X14')
bipartite_graph.add_edge('A9', 'X15')
bipartite_graph.add_edge('A9', 'X18')

bipartite_graph.add_edge('A10', 'X16')

bipartite_graph.print_empirical_study(verbose=False)
del bipartite_graph

The adjacency matrix for the bipartite graph is (rows->categories, columns->points):
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

The maximum matchings for this bipartite graphs are:
The complete list of edges is [('A1', 'X1'), ('A1', 'X2'), ('A1', 'X7'), ('A1', 'X12'), ('A2', 'X2'), ('A2', 'X4'), ('A2', 'X5'), ('A3', 'X5'), ('A4', 'X5'), ('A5', 'X7'), ('A5', 'X6'), ('A6', 'X8'), ('A6', 'X14'), ('A6', 'X17'), ('A7', 'X12'), ('A7', 'X9'), ('A7', 'X10'), ('A7', 'X11'), ('A8', 'X10'), ('A9', 

In [13]:
#Example 10
bipartite_graph = BipartiteGraph(8, 11)
bipartite_graph.add_edge('A1', 'X1')
bipartite_graph.add_edge('A1', 'X2')
bipartite_graph.add_edge('A1', 'X3')
bipartite_graph.add_edge('A1', 'X4')

bipartite_graph.add_edge('A2', 'X3')
bipartite_graph.add_edge('A2', 'X4')
bipartite_graph.add_edge('A2', 'X5')
bipartite_graph.add_edge('A2', 'X6')

bipartite_graph.add_edge('A3', 'X6')
bipartite_graph.add_edge('A3', 'X7')
bipartite_graph.add_edge('A3', 'X8')
bipartite_graph.add_edge('A3', 'X9')

bipartite_graph.add_edge('A4', 'X8')
bipartite_graph.add_edge('A4', 'X9')
bipartite_graph.add_edge('A4', 'X10')
bipartite_graph.add_edge('A4', 'X11')

bipartite_graph.add_edge('A5', 'X1')
bipartite_graph.add_edge('A5', 'X2')
bipartite_graph.add_edge('A5', 'X3')
bipartite_graph.add_edge('A5', 'X4')

bipartite_graph.add_edge('A6', 'X1')
bipartite_graph.add_edge('A6', 'X2')
bipartite_graph.add_edge('A6', 'X3')

bipartite_graph.add_edge('A7', 'X7')
bipartite_graph.add_edge('A7', 'X8')
bipartite_graph.add_edge('A7', 'X9')

bipartite_graph.add_edge('A8', 'X8')
bipartite_graph.add_edge('A8', 'X9')

bipartite_graph.print_empirical_study(verbose=False)
del bipartite_graph

The adjacency matrix for the bipartite graph is (rows->categories, columns->points):
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]

The maximum matchings for this bipartite graphs are:
The complete list of edges is [('A1', 'X1'), ('A1', 'X2'), ('A1', 'X3'), ('A1', 'X4'), ('A2', 'X3'), ('A2', 'X4'), ('A2', 'X5'), ('A2', 'X6'), ('A3', 'X6'), ('A3', 'X7'), ('A3', 'X8'), ('A3', 'X9'), ('A4', 'X8'), ('A4', 'X9'), ('A4', 'X10'), ('A4', 'X11'), ('A5', 'X1'), ('A5', 'X2'), ('A5', 'X3'), ('A5', 'X4'), ('A6', 'X1'), ('A6', 'X2'), ('A6', 'X3'), ('A7', 'X7'), ('A7', 'X8'), ('A7', 'X9'), ('A8', 'X8'), ('A8', 'X9')]

Searching matchings of size: 8
There are 3108105 combinations to check

number of matchings of size 8: 648

the best matchings are:
[('A1', 'X2'), ('A2', 'X6'), ('